In [1]:
from load_recommender_data import load_recommender_data

df_merge, anime_df, user_df =  load_recommender_data('user_data1.0.json')

sample = df_merge.sample(n=5000)
sample.shape


/home/dipesh/repositories/Codagami_Anime_Recommender/load_recommender_data.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  user_df = melt_series(df.data).to_frame().join(df.drop('data', 1))
/home/dipesh/repositories/Codagami_Anime_Recommender/env/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/home/dipesh/repositories/Codagami_Anime_Recommender/load_recommender_data.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

(5000, 8)

In [2]:
sample.dtypes

anime_id    float64
name         object
genre        object
episodes    float64
rating_x      int64
members       int64
user_id       int64
rating_y      int64
dtype: object

In [3]:
sample.head()

,anime_id,name,genre,episodes,rating_x,members,user_id,rating_y
30083,8425.0,Gosick,"[Drama, Historical, Mystery, Romance]",24.0,8,472247,49,0
39459,32093.0,Tanaka-kun wa Itsumo Kedaruge,"[Comedy, School, Slice of Life]",12.0,7,377418,89,7
48192,40936.0,Ore wo Suki nano wa Omae dake ka yo: Oretachi ...,"[Comedy, Romance, School]",1.0,7,94064,94,7
37256,28207.0,Ketsuekigata-kun! 2,[Comedy],12.0,6,4451,36,7
20694,38740.0,Bai She: Yuan Qi,"[Adventure, Demons, Drama, Fantasy, Historical...",1.0,7,5346,7,8


In [4]:
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds 

from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate

from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline

from surprise.prediction_algorithms import knns
from surprise.prediction_algorithms import SVD

from surprise.similarities import cosine, msd, pearson

from surprise import accuracy 
from surprise import Reader
from surprise import Dataset
from surprise import dataset

data = sample[['user_id', 'anime_id', 'rating_x']]
reader = Reader(line_format='user item rating', sep='')
anime_loaded_data = Dataset.load_from_df(data, reader)

#train_test_split
trainset, testset = train_test_split(anime_loaded_data, test_size=.2)

In [5]:
svd = SVD()
svd.fit(trainset)

In [6]:
predictions = svd.test(testset) #
accuracy.rmse(predictions)


RMSE: 2.2093


2.209298531208492

In [7]:
predictions[:10]


[Prediction(uid=66, iid=13629.0, r_ui=7.0, est=5, details={'was_impossible': False}),
 Prediction(uid=27, iid=9488.0, r_ui=7.0, est=5, details={'was_impossible': False}),
 Prediction(uid=62, iid=16035.0, r_ui=7.0, est=5, details={'was_impossible': False}),
 Prediction(uid=82, iid=27525.0, r_ui=7.0, est=5, details={'was_impossible': False}),
 Prediction(uid=13, iid=24415.0, r_ui=8.0, est=5, details={'was_impossible': False}),
 Prediction(uid=38, iid=37999.0, r_ui=8.0, est=5, details={'was_impossible': False}),
 Prediction(uid=68, iid=12031.0, r_ui=8.0, est=5, details={'was_impossible': False}),
 Prediction(uid=31, iid=31580.0, r_ui=7.0, est=5, details={'was_impossible': False}),
 Prediction(uid=33, iid=30364.0, r_ui=7.0, est=5, details={'was_impossible': False}),
 Prediction(uid=32, iid=42938.0, r_ui=9.0, est=5, details={'was_impossible': False})]

In [9]:
print("Number of users:", sample.user_id.nunique())


Number of users: 94


In [10]:
user = 5
item = 100
svd.predict(user, item)


Prediction(uid=5, iid=100, r_ui=None, est=5, details={'was_impossible': False})

In [11]:
params = {'n_factors': [20,50,100],
         'reg_all': [.02,.05, .10]}
gridsearch_svd1 = GridSearchCV(SVD, param_grid=params, n_jobs=-1, joblib_verbose=3)
gridsearch_svd1.fit(anime_loaded_data)

print(gridsearch_svd1.best_score)
print(gridsearch_svd1.best_params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.1s


{'rmse': 2.1925590335512815, 'mae': 2.0444000000000004}
{'rmse': {'n_factors': 20, 'reg_all': 0.02}, 'mae': {'n_factors': 20, 'reg_all': 0.02}}


[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    2.5s finished
